# Convergence tests

## Linear Advection with Constant Coefficient

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


#Local libraries
from basis_nodes import generate_lagrange_poly, generate_LGL_points
from SBP_matrices import first_order_D, first_order_P_Q
from RHS_1D_advection import generate_RHS_1D_advection
from SSPRK import SSPRK33

### Defining problem and exact solution to test convergence

In [ ]:
#Define physical domain
xlower = 0.
xupper = 2.

#Advection speed
c_advection = 1.

#Define initial conditions for convergence test
initial_c = lambda x: np.exp(-50*(x-1)**2)+1

#Define exact solution (Periodic boundary conditions)

def periodic_function(func, period, offset):
    return lambda x: func( ((x - offset) % period ) + offset )
def exact_sol(x,t):
    toeval = periodic_function(initial_c,xupper-xlower,0.)
    return toeval(x-c_advection*t)

### Different combinations of DOFs

In [ ]:
#Spatial refinement
h_refinement = np.array([5,10,20,40,80]) #Number of elements
p_refinement = np.array([3,4,5,6,7]) #Number of nodes per element

ref_level_h = len(h_refinement)
ref_level_p = len(p_refinement)

In [ ]:
#Computing errors
tfinal = 1.
CFL=0.8
error_list = [[] for i in range(ref_level_h)]
for idx_h in range(ref_level_h):
    #h refinement
    h = h_refinement[idx_h]
    for idx_p in range(ref_level_p):
        #p refinement
        p = p_refinement[idx_p]

        #Get RHS
        RHS_local, xgrid_local, xiLGL_local = generate_RHS_1D_advection(xlower=xlower,
        xupper=xupper, n_elements=h,order_x=p,
        c_advection=c_advection,BC="periodic",num_flux_type="upwind")

        #Compute time step
        dx = np.min(np.diff(xgrid_local[:p]))
        dxi = np.min(np.diff(xiLGL_local))
        dt = CFL*dx*dxi/c_advection/2

        #Compute numerical solution
        u0 = initial_c(xgrid_local)
        u_frames, t_eval = SSPRK33(fun=RHS_local, u0=u0, dt=dt, t0=0.,tfinal=tfinal, nframes=2)

        #Compute error
        u_numerical = u_frames[-1]
        #Compute exact solution
        u_exact = exact_sol(xgrid_local,t_eval[-1]) 
        #Compute error
        error = np.linalg.norm(u_numerical-u_exact,ord=2)/np.linalg.norm(u_exact,ord=2)
        error_list[idx_h].append(error)

In [ ]:
#Displaying results
df = pd.DataFrame(error_list)
df.columns = p_refinement
df.index = h_refinement
#Adding the proper headers
df.columns.name = "p"
df.index.name = "h"
#Computing the rate of convergence
df

### Fixed h=50

In [ ]:
#Spatial refinement
h_refinement = np.array([50]) #Number of elements
p_refinement = np.array([3,4,5,6,7]) #Number of nodes per element

ref_level_h = len(h_refinement)
ref_level_p = len(p_refinement)

#Computing errors
tfinal = 1.
CFL=0.8
error_list = [[] for i in range(ref_level)]
for idx_h in range(ref_level_h):
    #h refinement
    h = h_refinement[idx_h]
    for idx_p in range(ref_level_p):
        #p refinement
        p = p_refinement[idx_p]

        #Get RHS
        RHS_local, xgrid_local, xiLGL_local = generate_RHS_1D_advection(xlower=xlower,
        xupper=xupper, n_elements=h,order_x=p,
        c_advection=c_advection,BC="periodic",num_flux_type="upwind")

        #Compute time step
        dx = np.min(np.diff(xgrid_local[:p]))
        dxi = np.min(np.diff(xiLGL_local))
        dt = CFL*dx*dxi/c_advection/2

        #Compute numerical solution
        u0 = initial_c(xgrid_local)
        u_frames, t_eval = SSPRK33(fun=RHS_local, u0=u0, dt=dt, t0=0.,tfinal=tfinal, nframes=2)

        #Compute error
        u_numerical = u_frames[-1]
        #Compute exact solution
        u_exact = exact_sol(xgrid_local,t_eval[-1]) 
        #Compute error
        error = np.linalg.norm(u_numerical-u_exact,ord=2)/np.linalg.norm(u_exact,ord=2)
        error_list[idx_h].append(error)